In [1]:
import sys

sys.path.insert(0, '..')

In [3]:
from pathlib import Path
import numpy as np
from torch.utils.data import DataLoader
from core.multimodal.dataset import collate_fn, ASASSNVarStarDataset
from functools import partial
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import random
import numpy as np
import json
import os

In [4]:
random.seed(42)

In [14]:
datapath = Path('../data/asassn/')
ds = ASASSNVarStarDataset(datapath, mode='train', verbose=True, only_periodic=True,
                          merge_type='inner', recalc_period=False, prime=True, use_bands=['v', 'g'],
                          only_sources_with_spectra=False)

Opening v data files... Found 687695 sources. done.
Opening g data files... Found 378861 sources. done.
Removing duplicates for v band... Left with 675811. done.
Removing duplicates for g band... Left with 378823. done.
Removing objects that have class different from ['CWA', 'CWB', 'DCEP', 'DCEPS', 'DSCT', 'EA', 'EB', 'EW', 'HADS', 'M', 'ROT', 'RRAB', 'RRC', 'RRD', 'RVA', 'SR'] for v band... Left with 459022. done.
Removing objects that have class different from ['CWA', 'CWB', 'DCEP', 'DCEPS', 'DSCT', 'EA', 'EB', 'EW', 'HADS', 'M', 'ROT', 'RRAB', 'RRC', 'RRD', 'RVA', 'SR'] for g band... Left with 363971. done.
Merging bands... done. Now 212359 sources.
Removed non-periodic sources. Now 212359 sources.
Total: 212359, Train: 169887, Val: 21235, Test: 21237
Priming tarballs by doing initial scan... done.


In [15]:
total_flux_v = ds[0]['lcs'][0][0][:, 1]
total_flux_g = ds[0]['lcs'][0][1][:, 1]

In [16]:
indx = random.sample(range(1, len(ds)), 999)

In [17]:
for i in tqdm(indx):
    flux_v = ds[i]['lcs'][0][0][:, 1]
    flux_g = ds[i]['lcs'][0][1][:, 1]

    total_flux_v = np.concatenate((total_flux_v, flux_v))
    total_flux_g = np.concatenate((total_flux_g, flux_g))

100%|█████████████████████████████████████████| 999/999 [01:29<00:00, 11.20it/s]


In [18]:
len(total_flux_v), len(total_flux_g)

(525172, 520947)

In [19]:
total_flux_v.mean(), total_flux_v.std()

(60.85642597660034, 56.55015721187325)

In [20]:
total_flux_g.mean(), total_flux_g.std()

(10.698049152792896, 16.376429542424507)

In [21]:
res = {
    'v': {'mean': total_flux_v.mean(), 'std': total_flux_v.std()},
    'g': {'mean': total_flux_g.mean(), 'std': total_flux_g.std()}
}

In [22]:
with open(os.path.join(datapath, 'scales.json'), 'w') as file:
    json.dump(res, file)

In [23]:
ds.data_root / 'scales.json'

PosixPath('../data/asassn/scales.json')